# Checking enviroment states and action sets

In [1]:
import sys
import os.path
sys.path.append("..")
from PPO_Agent_env_model import * #PPO_Agent_v2 PPO_Agent_with_Guided_AC
from ENV_DETAILS import *
from RUN_TENSORBOARD import *

events_folder = "./logs_hyper"
main("./logs_hyper")

2024-06-07 14:52:07.165131: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num devices available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Selected port: 53767


2024-06-07 14:52:09.727176: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.10.1 at http://localhost:53767/ (Press CTRL+C to quit)


In [2]:
ENV = "MountainCarContinuous-v0"
SUCESS_CRITERIA_VALUE = ENV_DETAILS[ENV]["SUCESS_CRITERIA_VALUE"]
SUCESS_CRITERIA_EPOCH = ENV_DETAILS[ENV]["SUCESS_CRITERIA_EPOCH"]

EPISODES = ENV_DETAILS[ENV]["EPISODES"]

In [3]:

env = gym.make(ENV)
env

<TimeLimit<OrderEnforcing<Continuous_MountainCarEnv<MountainCarContinuous-v0>>>>

In [4]:
observation = env.reset()

# Initialize an empty list to store rewards
rewards = []

# Run episodes and collect rewards
for episode in range(100000):
    done = False
    while not done:
        observation, reward, done, info = env.step(env.action_space.sample())
        rewards.append(reward)

# Calculate normalization factor (e.g., mean or standard deviation)
mean_reward = np.mean(rewards)
std_dev_reward = np.std(rewards)
mean_reward, std_dev_reward

(92.15160670082223, 26.840920728403027)

In [5]:
s = env.observation_space.sample()
s

array([0.47566143, 0.03575775], dtype=float32)

In [6]:
env.observation_space, env.observation_space.shape[0]

(Box([-1.2  -0.07], [0.6  0.07], (2,), float32), 2)

In [7]:
s = env.reset()
s[0]

-0.5166163

In [8]:
env.action_space

Box(-1.0, 1.0, (1,), float32)

### Training/Hyperparam run

In [9]:
TUNING_TYPE = "BAYES"
HYPERPARAM_TUNING = True
writer= "logs_hyper/fit_PPO_deep_dive"

In [10]:
if HYPERPARAM_TUNING: 

    dir = r"Hyperparam_kt_ppo"
    project_name = "keras_tunning_ppo_deep_dive"

    tuner = kt.BayesianOptimization(
            MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/ppo_deep_dive/", evaluation_epoch = env._max_episode_steps, training_steps = 1000000,
                sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value= SUCESS_CRITERIA_VALUE,
                discount_min = 0.96, discount_max = 0.99, 
                #discount= 0.99, # WAS THIS
                gae_factor = 0.95,  # WAS THIS
                #gae_min = 0.92, gae_max = 0.96, 
                policy_clip =0.2,
                lr_actor_crit_min = 0.0005, lr_actor_crit_max = 0.005,
                #entropy_factor = 0.05,  # WAS THIS
                entropy_factor_min = 0.001, entropy_factor_max = 0.2,
                lr_model_min = 0.0001, lr_model_max =  0.001, kl_divergence_target = 0.1, ##### Try 0.1 after##############
                #dense_layers = [42,62],
                dense_min = 32, dense_max = 128, #WAS THIS ONE
                environment_name=ENV, num_layers_act = 2, #max_num_layers_act = 2
                num_layers_model = 1,  # WAS 2
                training_epoch = 1,
                memory_size = env._max_episode_steps, 
                normalize_reward=False, normalize_advantage= False,
                scaling_factor_reward = 0.1
                #memory_size_max= env._max_episode_steps
                ),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 60,
            # distribution_strategy= strategy,
            directory=dir,
            project_name=project_name,
            #seed=0
        )
    tuner.search(x=[0], y=[1])
else : 
    
        print("Acquiring parameters ....")

        dense_units_act_crit = [128,64]
        num_layers_actor_critic = len(dense_units_act_crit)
        num_layer_m = 1
        dense_units_model = [32]

        model = run_training(training_steps = 120000,
                            discount = 0.99, 
                            dense_units_act_crit = dense_units_act_crit,  
                            dense_units_model = dense_units_model,  
                            num_layer_a_c = num_layers_actor_critic,  
                            num_layer_m = num_layer_m, 
                            writer = writer, 
                            environment_name = ENV, 
                            return_agent = True, 
                            lr_actor_critic= 0.0001,  
                            lr_model = 0.001,
                            sucess_criteria_epochs=SUCESS_CRITERIA_EPOCH, 
                            sucess_criteria_value = SUCESS_CRITERIA_VALUE, 
                            gae_lambda = 0.95, 
                            entropy_coeff = 0.05, 
                            policy_clip = 0.2, training_epoch = 20, 
                            scaling_factor_reward = 0.1, 
                            kl_divergence_target = 0.1,
                            memory_size =  env._max_episode_steps,
                            normalize_reward=False, normalize_advantage= False, use_mlflow = False)


Reloading Tuner from Hyperparam_kt_ppo/keras_tunning_ppo_deep_dive/tuner0.json


In [11]:
exploration_tech = "soft"
hyperparam_combination=[]

for trials in tuner.oracle.get_best_trials(num_trials=3):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

Trial id :33 | Score :43.05638682043875 --> {'discount': 0.96, 'lr_actor_critic': 0.004567871160273846, 'lr_model': 0.0002748035222914486, 'entropy_coeff': 0.0785056939938031, 'dense_units_act_crit_0': 52, 'dense_units_act_crit_1': 119, 'n_dense_layers_model0': 51}
Trial id :35 | Score :21.429890302897984 --> {'discount': 0.96, 'lr_actor_critic': 0.004567871160273846, 'lr_model': 0.0002748035222914486, 'entropy_coeff': 0.0785056939938031, 'dense_units_act_crit_0': 52, 'dense_units_act_crit_1': 119, 'n_dense_layers_model0': 51}
Trial id :23 | Score :-1.8389723276779273 --> {'discount': 0.97, 'lr_actor_critic': 0.0036705858845969106, 'lr_model': 0.0001, 'entropy_coeff': 0.001, 'dense_units_act_crit_0': 101, 'dense_units_act_crit_1': 103, 'n_dense_layers_model0': 88}


In [13]:
env_model = tuner.get_best_models()[0]
val_env = gym.make(ENV)#, render_mode = "rgb_array"
final_rewards = final_evaluation(env_model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./ppo_intrinsic_"+exploration_tech+"_video.mp4")

Trial number :  60
